In [103]:
import requests
import icalendar
import arrow

from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, BooleanAttribute


In [104]:
def get_ipl_calendar():
    r = requests.get("http://www.espncricinfo.com/ci/content/series/1131611.ics?template=ical")
    return r.content

In [105]:
class MatchInfoModel(Model):
    """
    """
    class Meta:
        table_name = 'MatchInfoTable'
        region = 'us-east-1'
    match_id = NumberAttribute(hash_key=True)
    home_team = UnicodeAttribute()
    away_team = UnicodeAttribute()
    start_time = UnicodeAttribute()
    bets_processed = BooleanAttribute(default=False)


In [106]:
calendar = icalendar.Calendar.from_ical(get_ipl_calendar())

def get_match_id(component):
    return int(component['UID'].split('_')[0])

def get_home_team(component):
    first_part = component['SUMMARY'].split(" v ")[0]
    return " ".join(first_part.split()[2:]) or "TBD"

def get_away_team(component):
    return component['SUMMARY'].split(" v ")[1]

def get_start_time(component):
    return component['DTSTART'].to_ical().decode('utf-8')
    

for match in calendar.subcomponents:
    match_info = MatchInfoModel()
    match_info.match_id = get_match_id(match)
    match_info.home_team = get_home_team(match)
    match_info.away_team = get_away_team(match)
    match_info.start_time = get_start_time(match)
    match_info.save()
